In [1]:
!pip install gdown

In [2]:
import gdown

file_id = "1uvSVd-9T2eDvaFK-PvxWEFSgiwmm9-3Q"
url = f"https://drive.google.com/uc?id={file_id}"
gdown.download(url, output="my_file.zip", quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1uvSVd-9T2eDvaFK-PvxWEFSgiwmm9-3Q
From (redirected): https://drive.google.com/uc?id=1uvSVd-9T2eDvaFK-PvxWEFSgiwmm9-3Q&confirm=t&uuid=d9aa2c8f-e658-45ad-9c03-c8bfb59fd696
To: /kaggle/working/my_file.zip
100%|██████████| 4.09G/4.09G [01:45<00:00, 39.0MB/s]


'my_file.zip'

In [3]:
import zipfile
import os

def extract_zip(zip_path, extract_to='.'):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
        print(f"✅ Extracted '{zip_path}' to '{os.path.abspath(extract_to)}'")


zip_file = "/kaggle/working/my_file.zip"  
extract_folder = "/kaggle/working/"  
os.makedirs(extract_folder, exist_ok=True)

extract_zip(zip_file, extract_folder)

✅ Extracted '/kaggle/working/my_file.zip' to '/kaggle/working'


In [4]:
import numpy as np
import pandas as pd

for dirname, _, filenames in os.walk('/kaggle/working'):
    print(dirname)

/kaggle/working
/kaggle/working/.virtual_documents
/kaggle/working/__MACOSX
/kaggle/working/__MACOSX/Dataset
/kaggle/working/__MACOSX/Dataset/Test
/kaggle/working/__MACOSX/Dataset/Test/Blocked
/kaggle/working/__MACOSX/Dataset/Test/Normal
/kaggle/working/__MACOSX/Dataset/Train
/kaggle/working/__MACOSX/Dataset/Train/Blocked
/kaggle/working/__MACOSX/Dataset/Train/Normal
/kaggle/working/Dataset
/kaggle/working/Dataset/Test
/kaggle/working/Dataset/Test/Blocked
/kaggle/working/Dataset/Test/Normal
/kaggle/working/Dataset/Train
/kaggle/working/Dataset/Train/Blocked
/kaggle/working/Dataset/Train/Normal


In [5]:
import torch
import torch.nn as nn
from torchvision import models

class Teacher(nn.Module):
    def __init__(self, pretrained=True):
        super(Teacher, self).__init__()

        self.base_model = models.resnet18(pretrained=pretrained)
        
        num_features = self.base_model.fc.in_features
        self.linear = nn.Linear(1000, 512)  

        self.classifier = nn.Sequential( 
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(0.5),
            nn.Linear(256, 2)
        )
        
    def forward(self, x):
        x = self.base_model(x)
        # print("o1",x.shape)
        x = self.linear(x)
        # print("o1",x.shape)
        conv_output = x
        x = self.classifier(x)
        # print("o2",x.shape)
        return x,conv_output

In [7]:
from torchsummary import summary
import torch

model = Teacher().to("cuda")
summary(model, input_size=(3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           9,408
       BatchNorm2d-2           [-1, 64, 64, 64]             128
              ReLU-3           [-1, 64, 64, 64]               0
         MaxPool2d-4           [-1, 64, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
              ReLU-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
             ReLU-10           [-1, 64, 32, 32]               0
       BasicBlock-11           [-1, 64, 32, 32]               0
           Conv2d-12           [-1, 64, 32, 32]          36,864
      BatchNorm2d-13           [-1, 64, 32, 32]             128
             ReLU-14           [-1, 64,

In [8]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # ResNet expects 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

train_dir = "/kaggle/working/Dataset/Train"
test_dir = "/kaggle/working/Dataset/Test"

# Load datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Print dataset info
print("Train samples:", len(train_dataset))
print("Test samples:", len(test_dataset))
print("Classes:", train_dataset.classes)

Train samples: 12000
Test samples: 2000
Classes: ['Blocked', 'Normal']


In [9]:
def train_model(model, train_loader, criterion, optimizer, device, epochs=5):
    model.to(device)
    model.train()

    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)                # [B, 2]
            conv_output = outputs[1]
            outputs = outputs[0]
            loss = criterion(outputs, labels)      # labels: [B], values: 0 or 1
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        accuracy = correct / total
        avg_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{epochs}] | Loss: {avg_loss:.4f} | Accuracy: {accuracy:.4f}")
        print(conv_output.shape)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "mps")
model = Teacher(pretrained=True)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Train
train_model(model, train_loader, criterion, optimizer, device, epochs=5)

Epoch [1/5] | Loss: 0.0352 | Accuracy: 0.9887
torch.Size([32, 512])
Epoch [2/5] | Loss: 0.0116 | Accuracy: 0.9968
torch.Size([32, 512])
Epoch [3/5] | Loss: 0.0056 | Accuracy: 0.9983
torch.Size([32, 512])
Epoch [4/5] | Loss: 0.0042 | Accuracy: 0.9987
torch.Size([32, 512])
Epoch [5/5] | Loss: 0.0019 | Accuracy: 0.9995
torch.Size([32, 512])


In [12]:
from sklearn.metrics import classification_report

def test_model(model, test_loader, device, class_names):
    model.eval()
    model.to(device)
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            outputs = outputs[0]
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))
    
test_model(model, test_loader, "cuda", class_names=train_dataset.classes)


Classification Report:
              precision    recall  f1-score   support

     Blocked       1.00      0.97      0.98      1000
      Normal       0.97      1.00      0.98      1000

    accuracy                           0.98      2000
   macro avg       0.98      0.98      0.98      2000
weighted avg       0.98      0.98      0.98      2000



In [13]:
from IPython.display import FileLink
torch.save(model.state_dict(), "teacher-MSE.pth")
FileLink("teacher-MSE.pth")

/kaggle/working/teacher-MSE.pth